In [1]:
"""
you need [git clone https://github.com/comicom/hand-gesture-recognition/tree/master]
And then, replace "recognizer.ipynb" to control folder or update PATH in this code
"""

from tensorflow.keras.models import load_model, Model
from tensorflow.keras import layers
import cv2
import numpy as np
import tensorflow as tf

BASE_PATH = "../model"
MODEL_PATH = "/keras/hdf5/default_model.h5"
WEIGHT_PATH = "/keras/weight/default_weight.ckpt"
LABEL = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']
# FOR SIGN-LANGUAGE
#LABEL = ['A','B','C','D','E','F','G','H','I','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y'] 

In [2]:
class kerasModel:
    
    def __init__ (self, base_path=BASE_PATH, model_path=MODEL_PATH, weight_path=WEIGHT_PATH):
        self.model = load_model(base_path + model_path,compile=False)
        self.weight = self.model.load_weights(base_path + weight_path)
        self.output_shape = [layer.output_shape for layer in self.model.layers[:][:]]
        self.model.summary()
        print(base_path + model_path)
        print(base_path + weight_path)

    def state(self):
        layer_outputs = [layer.output for layer in self.model.layers[:]]
        layer_names = [layer.name for layer in self.model.layers[:][:]]
        layer_output_shape = [layer.output_shape for layer in self.model.layers[:][:]]
        output_shape = [output_shape for output_shape in self.output_shape[-1][1:]]
        print("layer outputs Check: ", [i for i in layer_outputs])
        print("layer names Check: ", layer_names)
        print("layer output_shape Check: ", layer_output_shape[:][:])
        
    def prediction(self,image):
        size = (self.output_shape[0][-1][1],self.output_shape[0][-1][2])
        output_shape = [output_shape for output_shape in self.output_shape[-1][1:]]
        resizedImage = np.expand_dims(cv2.resize(image, size),axis=0)
        if resizedImage.ndim != 4:
            resizedImage = np.expand_dims(resizedImage,axis=3)
        predict = self.model.predict(resizedImage)
        
        return predict

In [5]:
class realtimeRecognition:
    def __init__ (self, camID=0, base_path=BASE_PATH, model_path=MODEL_PATH, weight_path=WEIGHT_PATH):
        self.camID = camID
        self.cap = cv2.VideoCapture(self.camID)
        self.bg = None
        self.bgState = False
        self.kernel = np.ones((3,3),np.uint8)
        self.positions = {
            "hand_pose" : (15,40), #hand pose text
            "fps" : (15,20), #fps counter
            "null_pos" : (200,200) #used as null point for mous control
        }
        self.ROI = {
            "top" : 10,
            "right" : 350,
            "bottom" : 255,
            "left" : 590,
            "COLOR" : (0,255,0)
        }
        self.foreground_display = []
        self.mask_img = []
        self.roi = []
        self.h = []
        self.model = kerasModel(base_path=base_path, model_path=model_path, weight_path=weight_path)
    
    def __del__(self):
        self.cap.release
        cv2.destroyAllWindows()
    
    #Execute programe
    def run(self):
        # Capture > Process > Display loop
        while True:
            ### 1. Capture
            self.rect, self.frame = self.cap.read()
            self.frame = cv2.flip(self.frame, 1)
            self.display = self.frame.copy()
            ### 2. Process
            self.roi = self.frame[self.ROI["top"]:self.ROI["bottom"],
                                  self.ROI["right"]:self.ROI["left"]]
            self.foreground_display, self.mask_img = self.capBackground(self.frame)
  
            ### 3. Display
            cv2.rectangle(self.display,
                          (self.ROI["left"],self.ROI["top"]),
                          (self.ROI["right"],self.ROI["bottom"]),
                          self.ROI["COLOR"], 2)
            cv2.imshow("display",self.display)
            
            if self.foreground_display is not None:
                cv2.imshow("basic",self.roi)
                cv2.imshow("forground_display", self.foreground_display)
                cv2.imshow("mask_img", self.mask_img)
                self.h = self.roi*((cv2.cvtColor(self.foreground_display,cv2.COLOR_GRAY2RGB)/255).astype("uint8"))
                cv2.imshow("masked image",self.h)
                #cv2.imshow("test",(cv2.cvtColor(self.foreground_display,cv2.COLOR_GRAY2RGB)/255).astype("uint8"))
            
            ## 4. Free up memory
            k = cv2.waitKey(1)
            
            self.waitK(k)
    
    #Predict using Keras
    def kerasPrediction(self,image):
        #prediction
        predict = self.model.prediction(image)
        predi = predict.argmax()
        result = LABEL[predi]
        
        return result
        
    #Capture background for using find the difference between the background frame and current frame 
    def capBackground(self,image):
        if self.bg is None:
            cv2.putText(self.display,
                        "Press 'r' to reset the foreground extraction.",
                        self.positions["hand_pose"],
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.75, (0, 127, 64), 2)
            foreground_display = None
            mask_img = None
        else :
            foreground, mask, mask_img = self.extractForeground(image)
            foreground_display = foreground.copy()
        
        return foreground_display, mask_img
        
            
    def extractForeground(self,image):
        # Find the absolute difference between the background frame and current frame
        diff = cv2.absdiff(self.bg, image)
        mask = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
        
        #threshold the mask
        _, thresh = cv2.threshold(mask, 15, 255, cv2.THRESH_BINARY)
        
        #img preprocessing
#         opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, self.kernel)
#         closing = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, self.kernel)
        ero = cv2.erode(thresh,np.ones((3,3)))
        dil = cv2.dilate(ero,np.ones((5,5)))
        ###Helper function for applying a mask to an array
        forground = dil[self.ROI["top"]:self.ROI["bottom"],
                            self.ROI["right"]:self.ROI["left"]]
        mask_img = mask[self.ROI["top"]:self.ROI["bottom"],
                        self.ROI["right"]:self.ROI["left"]]
        return forground, mask, mask_img
    
    #Capture training images
    def capImage(self,num):
        if num == 0:
            #capture single image
            print("capImage 0: single image")
        elif num == 1:
            #capture mutiple images
            print("capImage 0: mutiple image")
        else:
            print("realtimeRcognition.capImage <<ERROR>>")
    
    #WaitKey
    def waitK(self,k):
        if k == ord("q"):
            self.cap.release()
            cv2.destroyAllWindows()
        elif k == ord("r"):
            self.bg = self.frame.copy()
            self.bgState = True
            print("realtimeRecognizer.extractForeground OK")
            print("realtimeRecognizer.capBackground OK")
#         elif k == ord("c"):
#             self.capImage(0)
#         elif k == ord("m"):
#             self.capImage(1)
        elif k == ord("p"):
            if self.bgState == True :
                if self.model.output_shape[0][-1][-1] == 3 :
                    result = self.kerasPrediction((self.h)/255.)
                    result = self.kerasPrediction((self.roi)/255.)
                elif self.model.output_shape[0][-1][-1] == 1 :
                    self.roi = cv2.cvtColor(self.roi,cv2.COLOR_RGB2GRAY)
                    result = self.kerasPrediction(self.roi/255.)
                else :
                    print(self.model.output_shape[0][-1],self.model.output_shape[0])
                    result = "error!"
                    
                print("result:",result)

            else :
                print("Press r first")


In [ ]:
if __name__ == "__main__":
    #recognizer = realtimeRecognition()
    recognizer = realtimeRecognition(model_path="/keras/hdf5/handgesture-raw-200214-28x28x3.h5",weight_path="/keras/weight/handgesture-raw-200214-28x28x3-0004.ckpt")
    recognizer.run()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 64)        1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               204928